<a href="https://colab.research.google.com/github/takatakamanbou/ML/blob/main/ex02noteB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ML ex02noteB

<img width=72 src="https://www-tlab.math.ryukoku.ac.jp/~takataka/course/ML/ML-logo.png"> [この授業のウェブページ](https://www-tlab.math.ryukoku.ac.jp/wiki/?ML/2022)


----
## 回帰のための教師あり学習(2) 平面の当てはめ
----

----
### 準備

Google Colab の Notebook では， Python というプログラミング言語のコードを動かして計算したりグラフを描いたりできます．
Python は，機械学習・人工知能やデータサイエンスの分野ではメジャーなプログラミング言語ですが，それを学ぶことはこの授業の守備範囲ではありません．以下の所々に現れるプログラムっぽい記述の内容は，理解できなくて構いません．

以下，コードセルを上から順に実行してながら読んでいってね．

In [ ]:
# 準備あれこれ
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn
seaborn.set()

今回新たに使うデータの準備をします．今回使うのは，機械学習の学習や実験に使えるデータセットを収集している [UCI Machine Learning Repository](https://archive.ics.uci.edu/ml/index.php) というサイトにある [Wine Quality Data Set](https://archive.ics.uci.edu/ml/datasets/wine+quality) というもの（のうちの赤ワインのデータ）です．

In [ ]:
# データを読み込む
dfWine = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv', header=0, sep=';')
dfWineL = dfWine[:1000] # 最初の1000個を学習データに，
dfWineT = dfWine[1000:] # 残り1000個をテストデータにする

# 学習データの最初の10個を表示させてみる
dfWineL[:10]

このデータは，とあるワインの物理化学的な性質を表す11種類の量（'pH'など）と，そのワインの品質を表す数値（'quality'）から成っています．'quality'は 0 から 10 の整数値です．

----
### 平面当てはめの問題設定



上記のデータに対して，11種類の量から品質を予測するという問題を考えてみます．
「ゴリゴリ君」のときは，気温 $x$ からアイス売上数 $y$ を予測するために $y = ax+b$ という式を立てましたが，このワインのデータの場合は $x$ に相当する数値が複数あります．

このような問題は，$x_1, x_2, \ldots, x_D$ という $D$ 個の値（いまは $D = 11$）から一つの値 $y$ を予測する問題ということができます．このとき，$x_1, \ldots , x_D$ から $y$ を予測する式として単純なのは，

$$
y = w_0 + w_1x_1 + w_2x_2 + \cdots + w_Dx_D \qquad (1)
$$

という形のものです．入力が1つの場合は直線の式でパラメータが2つだったのに対して，この式ではパラメータが $w_0, w_1, \ldots , w_D$ の $(D+1)$ 個あります．

式(1)は，$D=1$ のときは2次元平面上の直線，$D=2$ のときは3次元空間中の平面となり，一般の $D$ については，$(D+1)$次元空間中の $D$ 次元超平面となります．
以下の図は，2次元のデータに対する平面当てはめの例です．個々の青い点は，$(x_1, x_2, そのときの y の正解)$ を表し，赤いメッシュがこれらのデータに当てはめた平面を表しています．



<img src="https://www-tlab.math.ryukoku.ac.jp/~takataka/course/ML/planefitting.png" width="50%">


入力が1つ（1次元）の直線当てはめの問題では，最小二乗法によって最適な（二乗誤差の和が最小となる）パラメータを求めました．入力がD個（$D$次元）の場合も同様に，最小二乗法によってデータに当てはまる（超）平面のパラメータを求められます．


以下，最小二乗法による平面当てはめの問題設定を整理して説明します．ただし，式の見通しをよくするために，$1, x_1, x_2, \ldots, x_D$ という $(D+1)$ 個の値をならべた $(D+1)$ 次元ベクトルを 
$$
\mathbf{x} = (1, x_1, x_2, \ldots, x_D)
$$
と表し（先頭に$1$があることに注意），$(D+1)$個のパラメータをならべた $(D+1)$ 次元ベクトルを 
$$
\mathbf{w} = (w_0, w_1, w_2, \ldots, w_D)
$$
と表すことにします．こうしておくと，式(1)を
$$
y = f(\mathbf{x}) = \mathbf{w}\cdot\mathbf{x} \qquad (2)
$$
と簡単な形で書けます（注）．

※注: 記号$\cdot$は普通のベクトル同士の内積です．$\mathbf{w}\cdot\mathbf{x} = w_0 + w_1x_1+\ldots+w_Dx_D$.



**［最小二乗法による平面当てはめの問題設定］**

$D$ 個の値 $x_1, x_2, \ldots, x_D$ から $y$ の値が決まるようなデータがあり，それらの間の関係として式(2)が仮定できるとする．$N$個のデータ
$$
(\mathbf{x}_1, y_1), (\mathbf{x}_2, y_2),\ldots , (\mathbf{x}_N, y_N)
$$
が与えられる．ただし，$\mathbf{x}_n$は上述のように1と$D$個の値をならべた$(D+1)$次元ベクトルであり，$y_n$ はこのデータに対する $y$ の値の正解である（$n=1,2,\ldots, N$）．

このとき，$f(\mathbf{x}_n)$ とその正解の値 $y_n$ との間の二乗誤差の和
$$
\sum_{n=1}^{N}(y_n - f(\mathbf{x}_n))^2　=\sum_{n=1}^{N}(y_n - \mathbf{w}\cdot\mathbf{x}_n)^2 \qquad (3)
$$
を最小にするパラメータ $\mathbf{w}$ を求めたい．

----
### 平面当てはめの最小二乗法の解


式(3)を最小にするパラメータ $\mathbf{w} = (w_0, w_1, \ldots, w_D)$ は，直線当てはめと同様に「正規方程式」と呼ばれる連立方程式の解として求まります．以下，正規方程式がどんなものになるかを（証明抜きで）示します．


個々の学習データ（の先頭に1を付け足したもの）$\mathbf{x}_n$ を $(D+1)\times 1$ 行列（列ベクトル）とみなし，これをならべた行列を $X$ と表記します．つまり $X$は
$$
X = (\mathbf{x}_1\ \mathbf{x}_2\ \cdots\ \mathbf{x}_N)
$$
という $(D+1)\times N$ の行列です．また，正解の値 $y_n$ をならべた $1 \times N$ 行列を $Y$ と表記します．
$$
Y = (y_1\ y_2\ \ldots\ y_N)
$$
このとき，正規方程式は次式のようになります．
$$
XX^{\top}\mathbf{w} = XY^{\top} \qquad (4)
$$
ただし，$\mathbf{w}$ はパラメータをならべた $(D+1)\times 1$ 行列です．
この式の形からわかるように，正規方程式は $(D+1)$元の連立方程式です．

したがって，与えられた学習データから行列 $X, Y$ を作り，$XX^{\top}$ と $XY^{\top}$ を計算して式(4)を解けば，二乗誤差の和を最小とするパラメータ $\mathbf{w}$ が求まります（正規方程式がこのようになることを示すのは後の節に回します）．



----
### ワインの品質予測

ワインのデータを使って実際に平面当てはめの実験をやってみましょう．


In [ ]:
# 'quality' を除いた 11 種類の値を取り出す
X_raw = dfWineL.drop(columns='quality').to_numpy()
# 1 を付け足して (D+1)xN 行列 X をつくる
X = np.vstack((np.ones(len(X_raw)), X_raw.T))
print(X.shape)
print(X)

In [ ]:
# 正解の値をならべたベクトル Y をつくる
Y = dfWineL['quality'].to_numpy()
print(Y.shape)
print(Y)

In [ ]:
# 正規方程式を解く
XXT = X @ X.T  # 正規方程式の左辺の(D+1)x(D+1)の行列
XYT = X @ Y    # 正規方程式の右辺の(D+1)x1の行列
w = np.linalg.solve(XXT, XYT) # 連立方程式を解く
print(f'{len(w)}個のパラメータの値は')
print(w)

学習データに対して二乗誤差の和を最小にする12個のパラメータ $\mathbf{w} = w_0, w_1, \ldots, w_{11}$ の値が得られました．

このパラメータでどの程度うまくワインの品質を予測できるか，いくつかのデータで試してみましょう．

In [ ]:
# サンプルの番号
idx = [517, 38, 400, 100, 369]

# それぞれの入力，出力（予測値），正解の値を表示
for n in idx:
    xvec = X[:, n]
    y_predicted = w @ xvec  # 予測値を求める
    print('入力:', end='[ ')
    for xx in xvec[1:]:
        print(f'{xx:>6.2f}', end=' ')
    print(']', end=' ')
    print(f'出力: {y_predicted:.2f} 正解: {Y[n]}')

それっぽい値を出しているのもあれば，大きく外しているのもありますね．

#### ★ やってみよう
上記の `idx = [ ... , 369]` の末尾に `832` を追加して `idx = [ ... , 369, 832]` としてセルを実行し直し，832 番のデータの品質の予測値を求めましょう．その値をノート等（紙媒体）にメモしておこう．

----
### ［補足］ 回帰分析？重回帰分析？

この授業で登場している直線当てはめおよび平面当てはめの方法は，「教師あり学習」の「回帰」のための方法として最も簡単なものです．これらの方法は，データ分析や統計学の分野では，**回帰分析**，**重回帰分析**という名前で知られています（注）．

※注: 変数（被説明変数）が1つなのが（単）回帰，2つ以上なのが重回帰．区別せずどちらも回帰分析ということもあります．

----
### ［発展］平面当てはめの最小二乗法の解の導出

この節では，平面当てはめの最小二乗法の解が式(4)の連立方程式の解として求まることを示します．
この節の内容は発展的な話題であり，この授業で必ず修得すべきものとはしません．興味・時間の余裕のあるひとは式を追いかけたり式変形を自分でやってみたりしてください．

(step1) 誤差関数 $E(\mathbf{w})$ を次のように定義します．
$$
E(\mathbf{w}) = \frac{1}{2}\sum_{n=1}^N (y_n - \mathbf{w}\cdot\mathbf{x}_n)^2 = \frac{1}{2}\sum_{n=1}^N (y_n - \mathbf{w}^{\top}\mathbf{x}_n)^2
$$

(step2) $E(\mathbf{w})$ が最小となるパラメータを求めるために，これを $w_d$ ($d = 0, 1, 2, \ldots, D$) のそれぞれで偏微分したものを $0$ と置いた式を立てます．

$$
\begin{aligned}
\frac{\partial E(\mathbf{w})}{\partial w_d} &= \sum_{n=1}^N (y_n - \mathbf{w}^{\top}\mathbf{x}_n) \frac{\partial}{\partial w_d}(y_n - \mathbf{w}^{\top}\mathbf{x}_n)   \\
&= \sum_{n=1}^N (y_n - \mathbf{w}^{\top}\mathbf{x}_n)\left( -\frac{\partial}{\partial w_d}(w_0+w_1x_{n,1}+\cdots + w_Dx_{n,D})\right) \\
&= \sum_{n=1}^N (y_n - \mathbf{w}^{\top}\mathbf{x}_n)(-x_{n,d}) = 0 \qquad (d = 0, 1, 2, \ldots, D)
\end{aligned}
$$

ここで，$x_{n,d}$ はベクトル $\mathbf{x}_{n}$ の要素であり，$x_{n,0} \equiv 1$ です．

(step3) 上記の式を整理します．最後の部分より
$$
\begin{pmatrix}
\sum x_{n,0}\mathbf{w}^{\top}\mathbf{x}_n \\
\vdots \\
\sum x_{n,D}\mathbf{w}^{\top}\mathbf{x}_n
\end{pmatrix} = 
\begin{pmatrix}
\sum x_{n,0}y_n \\ \vdots \\ \sum x_{n,D}y_n
\end{pmatrix}
$$
が得られます．上で定義した行列・ベクトルを用いると，この式の左辺は，
$$
\begin{aligned}
(左辺) &= \left( \sum \mathbf{x}_n\mathbf{x}_n^{\top} \right) \mathbf{w} \\
&= 
\begin{pmatrix}
\mathbf{x}_1 & \mathbf{x}_2 & \cdots & \mathbf{x}_N
\end{pmatrix}
\begin{pmatrix}
\mathbf{x}_1^{\top} \\ \vdots \\ \mathbf{x}_N^{\top}
\end{pmatrix}
\mathbf{w}
= XX^{\top}\mathbf{w}
\end{aligned}
$$
と変形できます．同様に，右辺は，
$$
(右辺) = \begin{pmatrix}
\mathbf{x}_1 & \mathbf{x}_2 & \cdots & \mathbf{x}_N
\end{pmatrix}
\begin{pmatrix}
y_1 \\ \vdots \\ y_N
\end{pmatrix} = XY^{\top}
$$
となります．

以上より，正規方程式が式(4)の形となることが示されます．
